In [ ]:
## 

In [1]:
import os
import re
import glob
import math
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import scanpy as sc
from scipy.spatial import KDTree

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-bygtib4p because the default path (/home/stereonote/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [ ]:
file_name = 'sample_tmp.boundary.1500'
sample = file_name.split('.')[0]
limitation = int(file_name.split('_')[-1])
output_path = f'/data/work/pathologicalRegion/get_boundarypos/expansion/result/{file_name}'
os.makedirs(output_path, exist_ok=True)

In [ ]:
#step1 
step1_path = f'{output_path}/step1'
os.makedirs(step1_path, exist_ok=True)

def extract(xy_csv, sample, imageName, output):
    dat = pd.read_csv(xy_csv, index_col=0)
    msk = np.array(Image.open(imageName)).astype(float) 
    colorCutoff = 245
    r1 = [dat['x'].min(), dat['x'].max()]
    r2 = [dat['y'].min(), dat['y'].max()]
    s = msk.shape
    ox = oy = 0 
    k = 1
    if s[0] != s[1] or s[0] != 1000:
        print(f"错误：{imageName} 图片尺寸不符合要求", file=sys.stderr)
        sys.exit()
    if r1[1]-r1[0] > r2[1]-r2[0]:
        k = (r1[1]-r1[0])/s[0] 
    else:
        k = (r2[1]-r2[0])/s[1]
    idx = []
    for i in range(len(dat)):
        x = int((dat['x'].iloc[i] - r1[0]) / k + ox)
        y = s[1] - int((dat['y'].iloc[i] - r2[0]) / k + oy)
    
        x = max(0, min(x, s[1]-1))
        y = max(0, min(y, s[0]-1))
    
        pixel_value = (msk[y, x, 0] + msk[y, x, 1] + msk[y, x, 2])/3
        idx.append(pixel_value >= colorCutoff)
    dat['boundary_expansion'] = idx
    
    plt.figure(figsize=(10, 10), dpi=100)
    plt.scatter(dat['x'], dat['y'], c=np.where(dat['boundary_expansion'], '#FFDD00', '#EEEEEE'), s=8, marker='.')
    plt.axis('off')                
    plt.gca().set_aspect('equal') 
    plt.savefig(f"{output}.sel.png", bbox_inches='tight', pad_inches=0) 
    plt.close()
    
    dat = dat[idx][['x', 'y', 'boundary_expansion']]
    dat.to_csv(f"{output}.sel.csv",index=False)

In [ ]:
## step1
num_list = list(range(100, limitation + 1, 100))
bin100_csv = f'/data/work/pathologicalRegion/get_boundarypos/expansion/xy_pos/{sample}_region.csv'
for num in num_list:
    imageName = f'/data/work/pathologicalRegion/boundaryExpansion/output/{file_name}/expanded_boundary_{num}um.png'
    output = f'{step1_path}/{sample}.{num}um'
    extract(bin100_csv, sample, imageName, output)

In [ ]:
# step2 For each expansion layer, output the encompassed coordinates
step2_path = f'{output_path}/step2'
os.makedirs(step2_path, exist_ok=True)

num_list = list(range(100, limitation + 1, 100))
previous_df = None
for i, num in enumerate(num_list):
    input_file = f'{step1_path}/{sample}.{num}um.sel.csv'
    output_file = f'{step2_path}/{num}_um.circle.csv'
    current_df = pd.read_csv(input_file)
    if i == 0:
        current_df.to_csv(output_file, index=False)
    else:
        merged = current_df.merge(previous_df, on=['x', 'y', 'boundary_expansion'], how='left', indicator=True)
        result_df = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)
        result_df = result_df.loc[:, ['x', 'y', 'boundary_expansion']]
        result_df.to_csv(output_file, index=False)
    previous_df = current_df

In [ ]:
# step3 merge region info
step3_path = f'{output_path}/step3'
os.makedirs(step3_path, exist_ok=True)
print(sample)
region_file = pd.read_csv(f'/data/work/pathologicalRegion/get_boundarypos/expansion/xy_pos/{sample}_region.csv',index_col=0)
region_file = region_file[region_file['pathologicalRegion'].isin(['Fibrosis','Regression'])]
num_list = list(range(100, limitation + 1, 100))
for i, num in enumerate(num_list):
    input_file = f'{step2_path}/{num}_um.circle.csv'
    pos_file = pd.read_csv(input_file)
    tmp_file = pd.merge(region_file, pos_file, on = ['x','y'], how='right')
    tmp_file['region'] = tmp_file['pathologicalRegion']
    output_file = f'{step3_path}/{sample}.{num}um.region.csv'
    tmp_file.to_csv(output_file, index=False)

P111_PT


In [ ]:
##final result
all_result = []
output_file = f'{output_path}/{file_name}.boundaryexp.region.csv'
for i, num in enumerate(num_list):
    input_file = f'{step3_path}/{sample}.{num}um.region.csv'
    region_data = pd.read_csv(input_file)
    final_data = region_data[['x', 'y', 'region']].copy()
    final_data['size'] = num
    all_result.append(final_data)
all_result_file = pd.concat(all_result, ignore_index=True)
all_result_file = all_result_file.dropna(subset=['region'])
all_result_file.to_csv(output_file, index=False)

In [8]:
print('ggg')

ggg
